# Grounding in LTN (continuation)

This tutorial explains how to ground connectives and quantifiers in LTN. It expects some familiarity with the first tutorial on grounding non-logical symbols (constants, variables, functions and predicates).

# 在LTN中的基础（续）

本教程解释了如何在LTN中基础化连接词和量词。它要求熟悉第一个关于基础化非逻辑符号（常量、变量、函数和谓词）的教程。


In [40]:
import ltn
import numpy as np
import torch

## Connectives

LTN supports various logical connectives. They are grounded using fuzzy semantics. The implementation of some of the most
common fuzzy semantics using PyTorch primitives is included in the `ltn.fuzzy_ops` module.

In general, we recommend using the following semantics for LTN connectives:
* the standard negation  $\lnot u = 1-u$;
* the product t-norm $u \land v = uv$;
* the product t-conorm (probabilistic sum) $u \lor v = u+v-uv$;
* the Reichenbach implication $u \rightarrow v = 1 - u + uv$;

where $u$ and $v$ denote two truth values in $[0,1]$.

For more details about the choice of the right fuzzy semantics for your task, we suggest reading the complementary notebook (2b-grounding_connectives.ipynb).

In LTN, creating a connective is really simple. It is possible to use the constructor `Connective()`, which takes as input a
unary or binary fuzzy connective semantics. It is possible to choice the semantics from the `ltn.fuzzy_ops` module.

In this tutorial, we use the connective fuzzy semantics suggested above.

### 连结词

LTN 支持多种逻辑连结词。它们使用模糊语义进行基础化。在 `ltn.fuzzy_ops` 模块中包含了使用 PyTorch 原语实现的一些最常见的模糊语义。

一般来说，我们推荐使用以下语义来定义 LTN 连结词：
* 标准否定 $\lnot u = 1-u$；
* 乘积 t-范数 $u \land v = uv$；
* 乘积 t-余范数（概率和）$u \lor v = u+v-uv$；
* 莱兴巴赫蕴涵 $u \rightarrow v = 1 - u + uv$；

其中 $u$ 和 $v$ 表示在 $[0,1]$ 区间内的两个真值。

关于为您的任务选择合适的模糊语义的更多细节，建议阅读补充笔记本（2b-grounding_connectives.ipynb）。

在 LTN 中，创建连结词非常简单。可以使用构造函数 `Connective()`，其输入是一元或二元模糊连结词语义。可以从 `ltn.fuzzy_ops` 模块中选择语义。

在本教程中，我们使用上面建议的连结词模糊语义。

In [41]:
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())

In particular, the wrapper `ltn.Connective` allows to use the operators with LTN formulas. Specifically, it takes care
of combining sub-formulas which have different variables appearing in them (the sub-formulas may have different dimensions
that need to be "broadcasted" in order to apply the connective).

In this example, we create two variables with different number of individuals, two constants, and a predicate measuring the similarity between two points in $\mathbb{R}^2$.

特别地，封装器 `ltn.Connective` 允许在 LTN 公式中使用运算符。具体来说，它负责组合在其中出现不同变量的子公式（子公式可能具有不同的维度，需要进行“广播”以应用连接词）。

在此示例中，我们创建了两个具有不同个体数量的变量、两个常量，以及一个测量 $\mathbb{R}^2$ 中两点相似性的谓词。

In [42]:
x = ltn.Variable('x', torch.randn((10, 2))) # 10 values in R² # 10个二维点
y = ltn.Variable('y', torch.randn((5, 2))) # 5 values in R² # 5个二维点

c1 = ltn.Constant(torch.tensor([0.5, 0.0]))
c2 = ltn.Constant(torch.tensor([4.0, 2.0]))

Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity # 用欧氏距离来衡量相似度
# torch.norm(x - y, dim=1) 计算两个点之间的欧氏距离。
# torch.exp(-torch.norm(x - y, dim=1)) 通过计算距离的负指数来衡量相似度，距离越小，相似度越高。

Eq(c1, c2).value

tensor(0.0178, device='cuda:0')

Now, we check the behavior of the logical connectives. It is easy to see that they behave according to the definition
written at the beginning of the tutorial.

Notice the shape printed in the last two lines of code. In the first one, since only variable x appears in the formula,
we have a shape of 5. The formula has been evaluated for each x.
In the second one, since both x and y appear in the formula, we have a shape of 10x5.
The formula has been evaluated for each combination of x and y.

现在，我们检查逻辑连接词的行为。很容易看出它们的行为符合教程开头所写的定义。

请注意代码最后两行打印的形状。在第一行中，由于公式中仅出现变量 x，因此形状为 5。公式已针对每个 x 进行评估。
在第二行中，由于公式中同时出现了 x 和 y，因此形状为 10x5。公式已针对每个 x 和 y 的组合进行评估。

In [43]:
Not(Eq(c1, c2)).value

tensor(0.9822, device='cuda:0')

In [44]:
Implies(Eq(c1, c2), Eq(c2, c1)).value

tensor(0.9825, device='cuda:0')

In [45]:
# Notice the dimension of the outcome: the result is evaluated for every x. # 结果是对每个x进行评估的
And(Eq(x, c1), Eq(x, c2)).shape()
print("Eq(x,c1):"+"\n")
print(Eq(x, c1).value)
print("Eq(x,c2):"+"\n")
print(Eq(x, c2).value)
print("And(Eq(x,c1),Eq(x,c2)):"+"\n")
print(And(Eq(x, c1), Eq(x, c2)).value)

Eq(x,c1):

tensor([0.1019, 0.4467, 0.5762, 0.5634, 0.0887, 0.1915, 0.2415, 0.2674, 0.7903,
        0.8973], device='cuda:0')
Eq(x,c2):

tensor([0.0118, 0.0246, 0.0121, 0.0197, 0.0023, 0.0036, 0.0160, 0.0048, 0.0199,
        0.0183], device='cuda:0')
And(Eq(x,c1),Eq(x,c2)):

tensor([0.0012, 0.0110, 0.0070, 0.0112, 0.0002, 0.0007, 0.0039, 0.0013, 0.0158,
        0.0166], device='cuda:0')


In [46]:
# Notice the dimensions of the outcome: the result is evaluated for every x and y. # 结果是对每个x和y进行评估的
# Notice also that y did not appear in the 1st argument of `Or`; # 注意y没有出现在Or的第一个参数中
# the connective broadcasts the results of its two arguments to match. # 连接器将其两个参数的结果进行广播以匹配
Or(Eq(x, c1), Eq(x, y)).shape()

print("Eq(x,c1):"+"\n")
print(Eq(x, c1).value)
print("Eq(x,y):"+"\n")
print(Eq(x, y).value)
print("Or(Eq(x,c1),Eq(x,y)):"+"\n")
print(Or(Eq(x, c1), Eq(x, y)).value)

Eq(x,c1):

tensor([0.1019, 0.4467, 0.5762, 0.5634, 0.0887, 0.1915, 0.2415, 0.2674, 0.7903,
        0.8973], device='cuda:0')
Eq(x,y):

tensor([[0.0204, 0.0396, 0.0036, 0.0736, 0.0352],
        [0.2490, 0.5287, 0.0532, 0.2000, 0.6830],
        [0.1112, 0.5838, 0.0557, 0.4471, 0.3934],
        [0.0892, 0.2179, 0.0189, 0.2664, 0.1950],
        [0.0318, 0.1734, 0.1782, 0.1705, 0.1126],
        [0.0251, 0.1412, 0.0255, 0.4954, 0.0883],
        [0.0433, 0.0937, 0.0083, 0.1562, 0.0829],
        [0.0435, 0.2706, 0.0511, 0.7203, 0.1633],
        [0.1504, 0.4525, 0.0388, 0.3040, 0.4117],
        [0.1322, 0.4237, 0.0365, 0.3350, 0.3661]], device='cuda:0')
Or(Eq(x,c1),Eq(x,y)):

tensor([[0.1202, 0.1374, 0.1051, 0.1679, 0.1335],
        [0.5844, 0.7392, 0.4761, 0.5573, 0.8245],
        [0.6232, 0.8236, 0.5997, 0.7656, 0.7429],
        [0.6022, 0.6585, 0.5716, 0.6796, 0.6485],
        [0.1176, 0.2467, 0.2511, 0.2440, 0.1913],
        [0.2117, 0.3056, 0.2121, 0.5920, 0.2629],
        [0.2743, 0.3126,

Notice the access to the `value` attribute or `shape()` method of the evaluation of the connective. This is done because LTN connectives return `LTNObject` instances, like it happens for predicates and functions.

## Quantifiers

LTN suppports universal and existential quantification. They are grounded using aggregation operators. We recommend using the following two operators:

- existential quantification ("exists"):
the generalized mean (`pMean`) $\mathrm{pM}(u_1,\dots,u_n) = \biggl( \frac{1}{n} \sum\limits_{i=1}^n u_i^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;
- universal quantification ("for all"):
the generalized mean of "the deviations w.r.t. the truth" (`pMeanError`) $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;

where $u_1,\dots,u_n$ is a list of truth values in $[0,1]$.

In LTN, creating a quantifier is really simple. It is possible to use the constructor `Quantifier()`, which takes as input
an aggregation semantics and a character indicating which type of quantification is associated to the quantifier ("e" for exists, "f" for forall).

In this example, we create the quantifiers using the fuzzy semantics proposed above.

请注意对连结词的评估结果中的 `value` 属性或 `shape()` 方法的访问。这是因为LTN连结词返回的是 `LTNObject` 实例，这种情况同样适用于谓词和函数。

## 量词

LTN支持全称量化和存在量化。它们使用聚合运算符进行基础化。我们推荐使用以下两种运算符：

- 存在量化（“存在”）：
广义均值（`pMean`） $\mathrm{pM}(u_1,\dots,u_n) = \biggl( \frac{1}{n} \sum\limits_{i=1}^n u_i^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;
- 全称量化（“所有”）：
广义均值的“真值偏差”（`pMeanError`） $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;

其中 $u_1,\dots,u_n$ 是 $[0,1]$ 范围内的一组真值。

在LTN中，创建量词非常简单。可以使用构造函数 `Quantifier()`，该函数接受一个聚合语义和一个表示量化类型的字符作为输入（“e” 表示存在，“f” 表示全称）。

在这个例子中，我们使用上述模糊语义创建量词。

In [47]:
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")

The wrapper `ltn.Quantifier` allows to use the aggregators with LTN formulas. It takes care of selecting the tensor (formula)
dimensions to aggregate, given some variables in arguments.

In this example, we create variables and predicate similar to the previous example on connectives.


包装器 `ltn.Quantifier` 允许在 LTN 公式中使用聚合器。它负责根据参数中的变量选择要聚合的张量（公式）维度。

在这个例子中，我们创建了与之前关于连接词的例子类似的变量和谓词。

In [48]:
x = ltn.Variable('x', torch.randn((10, 2))) # 10 values in R² # 10个二维点
y = ltn.Variable('y', torch.randn((5, 2))) # 5 values in R² # 5个二维点

Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity # 用欧氏距离来衡量相似度

Eq(x, y).shape()

torch.Size([10, 5])

Now, we apply some quantifiers to the formula, and we see how this effects the output and its shape.

In the first case, we have quantified on x, so the output has shape 5. This means we have removed the dimension related to x and only dimension related to y is left. The shape is 5 since y has 5 individuals.

In the other three cases, the output is a scalar since the quantification has been performed on both variables. This means the aggregation has been performed on both dimension, namely the dimension of x and the dimension of y.

现在，我们对公式应用一些量词，并观察这如何影响输出及其形状。

在第一个例子中，我们对 $ x $ 进行了量化，所以输出的形状是 5。这意味着我们移除了与 $ x $ 相关的维度，只剩下与 $ y $ 相关的维度。形状为 5，因为 $ y $ 有 5 个个体。

在另外三个例子中，输出是一个标量，因为量化已对两个变量进行。这意味着聚合已经在两个维度上执行，即 $ x $ 的维度和 $ y $ 的维度。

In [49]:
Forall(x, Eq(x, y)).shape()
print("Eq(x,y):"+"\n")
print(Eq(x, y).value)
print("Forall(x,Eq(x,y)):"+"\n")
print(Forall(x, Eq(x, y)).value)

Eq(x,y):

tensor([[0.4836, 0.8175, 0.3073, 0.0733, 0.4667],
        [0.4097, 0.3629, 0.2195, 0.1366, 0.8526],
        [0.3096, 0.5001, 0.2854, 0.0416, 0.2220],
        [0.2247, 0.2581, 0.2867, 0.0332, 0.1307],
        [0.2234, 0.0910, 0.2216, 0.4663, 0.1734],
        [0.2633, 0.1189, 0.2060, 0.5063, 0.2604],
        [0.1056, 0.0437, 0.1059, 0.5319, 0.0892],
        [0.1418, 0.2519, 0.1451, 0.0191, 0.1041],
        [0.1336, 0.1448, 0.1922, 0.0217, 0.0742],
        [0.2407, 0.0957, 0.2725, 0.3353, 0.1629]], device='cuda:0')
Forall(x,Eq(x,y)):

tensor([0.2449, 0.2341, 0.2217, 0.1897, 0.2198], device='cuda:0')


In [50]:
Forall([x, y], Eq(x, y)).value
print("Forall([x, y], Eq(x, y)):"+"\n")
print(Forall([x, y], Eq(x, y)).value)

Forall([x, y], Eq(x, y)):

tensor(0.2218, device='cuda:0')


In [51]:
Exists([x, y], Eq(x, y)).value

tensor(0.3036, device='cuda:0')

In [52]:
Forall(x, Exists(y, Eq(x, y))).value

tensor(0.2724, device='cuda:0')

Notice the access to the `value` attribute or `shape()` method of the evaluation of the quantifier. This is done because LTN quantifiers return `LTNObject` instances, like it happens for predicates, functions, and connectives.

It is important to observe that when the quantification is performed on a single variable, it is possible to give the variable
to the quantifier as it comes, instead, if the quantification has to be performed on more variables, it is required to give the variables to the quantifier
through a list, as it happens for the second and third example.

## Semantics for quantifiers

`pMean` semantics can be understood as a soft-maximum that depends on the hyper-paramer $p$:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `max`.

Similarly, `pMeanError` semantics can be understood as a soft-minimum:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `min`.

Intuitively, $p$ offers flexibility in writing more or less strict formulas, to account for outliers in the data depending on the application.

Note that different choices of $p$ could have strong implications during the training (see 2b-grounding_connectives.ipynb).

One can set a default value for $p$ when initializing the operator, or can use different values at each call of the operator.

In this example, we use the quantifiers with different values of the $p$ parameter.

In general, higher is $p$ and easier is the existential quantification to be satisfied, while harder is the universal quantification to be satisfied.

注意访问量词评估的 `value` 属性或 `shape()` 方法。这是因为 LTN 量词返回 `LTNObject` 实例，就像谓词、函数和连接词一样。

重要的是，当对单个变量进行量化时，可以直接将变量传递给量词；而如果对多个变量进行量化，则需要通过列表将变量传递给量词，就像在第二个和第三个示例中一样。

## 量词的语义

`pMean` 语义可以理解为依赖于超参数 $p$ 的软最大值：
- 当 $p \to 1$ 时：运算符趋向于 `mean`，
- 当 $p \to +\infty$ 时：运算符趋向于 `max`。

同样，`pMeanError` 语义可以理解为软最小值：
- 当 $p \to 1$ 时：运算符趋向于 `mean`，
- 当 $p \to +\infty$ 时：运算符趋向于 `min`。

直观地说，$p$ 提供了编写更严格或更宽松公式的灵活性，以根据应用程序处理数据中的离群值。

请注意，不同的 $p$ 选择在训练过程中可能有强烈的影响（参见 2b-grounding_connectives.ipynb）。

在初始化运算符时，可以设置 $p$ 的默认值，或者在每次调用运算符时使用不同的值。

在此示例中，我们使用不同 $p$ 参数值的量词。

一般来说，$p$ 值越高，存在量化（existential quantification）越容易满足，而全称量化（universal quantification）则越难满足。

In [53]:
Forall(x, Eq(x, c1), p=2).value

tensor(0.2147, device='cuda:0')

In [54]:
Forall(x, Eq(x, c1), p=10).value # todo:这里的p参数是怎么传递的

tensor(0.1744, device='cuda:0')

In [55]:
Exists(x, Eq(x, c1), p=2).value

tensor(0.2518, device='cuda:0')

In [56]:
Exists(x, Eq(x, c1), p=10).value

tensor(0.3834, device='cuda:0')

## Diagonal Quantification

Given 2 (or more) variables, there are scenarios where one wants to express statements about specific pairs (or tuples) only, such that the $i$-th tuple contains the $i$-th instances of the variables.
In other words, in some cases we do not want to evaluate a formula on all the possible combination of individuals of the variables involved in it.

We allow this using `ltn.diag` (diagonal quantification).

**Note**: diagonal quantification assumes that the variables have the same number of individuals.
This is intuitive since we need a one-to-one correspondence between the individuals of the variables involved in the quantification.

In simplified pseudo-code, the usual quantification would compute:
```python
for x_i in x:
    for y_j in y:
        results.append(P(x_i,y_j))
aggregate(results)
```
In contrast, diagonal quantification would compute:
```python
for x_i, y_i in zip(x,y):
    results.append(P(x_i,y_i))
aggregate(results)
```

We illustrate `ltn.diag` in the following setting:
- the variable $x$ denotes 100 individuals in $\mathbb{R}^{2\times2}$,
- the variable $l$ denotes 100 one-hot labels in $\mathbb{N}^3$ (3 possible classes),
- $l$ is grounded according to $x$ such that each pair $(x_i,l_i)$, for $i=0..100$ denotes one correct example from the dataset,
- the classifier $C(x,l)$ gives a confidence value in $[0,1]$ of the sample $x$ corresponding to the label $l$.

### 对角量化

给定两个（或更多）变量，有时我们希望只对特定的对（或元组）进行表述，即第 $i$ 个元组只包含第 $i$ 个变量实例。换句话说，在某些情况下，我们不希望对涉及的变量的所有可能组合进行公式评估。

我们可以使用 `ltn.diag`（对角量化）实现这一点。

**注意**：对角量化假设变量具有相同数量的个体。这是直观的，因为我们需要变量个体之间的一一对应关系来进行量化。

在简化的伪代码中，通常的量化计算如下：
```python
for x_i in x:
    for y_j in y:
        results.append(P(x_i,y_j))
aggregate(results)
```
相比之下，对角量化的计算如下：
```python
for x_i, y_i in zip(x,y):
    results.append(P(x_i,y_i))
aggregate(results)
```

我们在以下设置中演示 `ltn.diag`：
- 变量 $x$ 表示 $\mathbb{R}^{2\times2}$ 中的100个个体，
- 变量 $l$ 表示 $\mathbb{N}^3$ 中的100个一热编码标签（3个可能的类别），
- $l$ 根据 $x$ 进行基础化，使得每个对 $(x_i,l_i)$，对于 $i=0..100$，表示数据集中一个正确的示例，
- 分类器 $C(x,l)$ 给出样本 $x$ 对应于标签 $l$ 的置信度值在 $[0,1]$ 之间。

In [57]:
# The values are generated at random, for the sake of illustration. # 值是随机生成的，仅用于说明
# In a real scenario, they would come from a dataset. # 在实际情况下，它们将来自数据集
samples = torch.randn((100, 2, 2)) # 100 R^{2x2} values # 100个R^{2x2}值
labels = torch.randint(0, 3, size=(100,)) # 100 labels (class 0/1/2) that correspond to each sample # 100个标签（类0/1/2），对应于每个样本（这个的每个样本应该是一个2x2的张量，一共100个）
# 生成 100 个随机整数标签，每个标签的取值范围为 0 到 2，对应于每个样本。
onehot_labels = torch.nn.functional.one_hot(labels, num_classes=3)
# 将标签转换为 one-hot 编码，表示每个标签是 3 个类别中的一个。

x = ltn.Variable("x", samples)
l = ltn.Variable("l", onehot_labels)

class ModelC(torch.nn.Module):
    def __init__(self):
        super(ModelC, self).__init__()
        self.elu = torch.nn.ELU()
        self.softmax = torch.nn.Softmax(dim=1)
        self.dense1 = torch.nn.Linear(4, 5)
        self.dense2 = torch.nn.Linear(5, 3)

    def forward(self, x, l):
        x = torch.flatten(x, start_dim=1)
        x = self.elu(self.dense1(x))
        x = self.softmax(self.dense2(x))
        return torch.sum(x * l, dim=1)

C = ltn.Predicate(ModelC().to(ltn.device)) # C是一个量词，但是这个量词所表示的具体含义一开始不是确定的，而应该是在训练过程中确定的。或者说一开始效果不好，是在训练过程中逐渐优化的，从而实现最终的效果。能够输出一个x中的个体与一个l中的标签之间的置信度。

In [58]:
# print("samples:")
# print(samples) # 从输出中可见，samples是一个100x2x2的张量
# print("\n")
# 
# print("labels:")
# print(labels) # 从输出中可见，labels是一个100个元素的张量
# print("\n")

# print("onehot_labels:")
# print(onehot_labels) # 从输出中可见，onehot_labels是一个100x3的张量
# print("\n")

# print("x:")
# print(x) # 从输出中可见，x是一个Variable对象，它的free_vars属性是一个字典，包含了一个key为"x"的元素，对应的value是一个100x2x2的张量。这个张量是一个100个2x2的张量的集合。
# print("\n")

# print("l:")
# print(l) # 从输出中可见，l是一个Variable对象，它的free_vars属性是一个字典，包含了一个key为"l"的元素，对应的value是一个100x3的张量
# print("\n")

If some variables are marked using `ltn.diag`, LTN will only compute their "zipped" results (instead of the usual "broadcasting").

In other words, the formula will be evaluated only on specific tuples of individuals instead of being evaluated on all the possible combinations of individuals of the variables.

It is possible to observe that the shape of the first evaluation is 100x100. This happens because LTN generates all the possible
combinations of individuals of $x$ and $l$, and then applies the predicate.

Note how the shape changes after applying `ltn.diag`. This happens because the individuals of the two variables are taken in one-to-one correspondence and the predicate
is computed only on that specific tuples of individuals.

Notice how the `free_vars` attribute of the variables is changed after a diagonal setting has been set. It is possible to recognize
if a variable is in diagonal setting by checking if its label begins with "diag_". In particular, all the variables in the same
diagonal setting will share the same label.

It is possible to use `ltn.undiag` to obtain the opposite behavior. In other words, it removes the diagonal setting from the variables
and restore the usual LTN broadcasting for the variables. This is clarified from the last print.


如果一些变量使用 `ltn.diag` 标记，LTN 只会计算它们的“拉链式”结果（而不是通常的“广播”）。

换句话说，公式将只在特定的个体元组上进行评估，而不是在变量的所有可能个体组合上进行评估。

可以观察到，第一次评估的形状是 100x100。这是因为 LTN 生成了 $x$ 和 $l$ 的所有可能个体组合，然后应用谓词。

请注意，在应用 `ltn.diag` 后，形状是如何变化的。这是因为两个变量的个体是一对一对应的，并且谓词只在特定的个体元组上进行计算。

注意，当设置了对角线设置后，变量的 `free_vars` 属性是如何变化的。可以通过检查标签是否以 “diag_” 开头来识别变量是否处于对角线设置中。特别地，所有在同一对角线设置中的变量将共享相同的标签。

可以使用 `ltn.undiag` 来获得相反的行为。换句话说，它从变量中移除对角线设置并恢复变量的通常 LTN 广播。这一点从最后的打印可以得到明确说明。


In [59]:
print(C(x, l).shape()) # Computes the 100x100 combinations # 计算100x100个组合
ltn.diag(x, l) # sets the diag behavior for x and l # 设置x和l的diag行为
print(C(x, l).shape())# Computes the 100 zipped combinations # 计算100个“zipped”组合
print(x.free_vars)
print(l.free_vars)
ltn.undiag(x, l) # resets the normal behavior # 重置正常行为
print(C(x, l).shape()) # Computes the 100x100 combinations # 计算100x100的组合
print(x.free_vars)
print(l.free_vars)

torch.Size([100, 100])
torch.Size([100])
['diag_x_l']
['diag_x_l']
torch.Size([100, 100])
['x']
['l']


In [60]:
ltn.diag(x, l)
print("C(x,l):"+"\n")
print(C(x, l).value) # 只有对角线上的100个

ltn.undiag(x, l)
print("C(x,l):"+"\n") # 包含所有的100*100个组合
print(C(x, l).value)

C(x,l):

tensor([0.2301, 0.1613, 0.3276, 0.3254, 0.3168, 0.2481, 0.5278, 0.2290, 0.2261,
        0.2674, 0.4786, 0.2760, 0.3135, 0.2870, 0.2123, 0.4901, 0.2583, 0.4538,
        0.4175, 0.6281, 0.3391, 0.4289, 0.2428, 0.5173, 0.3292, 0.2918, 0.3534,
        0.2723, 0.2341, 0.3881, 0.2593, 0.2193, 0.2033, 0.5614, 0.2991, 0.3218,
        0.2156, 0.3660, 0.2416, 0.3908, 0.4106, 0.2863, 0.4884, 0.4458, 0.5536,
        0.2249, 0.1896, 0.2896, 0.3417, 0.2210, 0.5214, 0.3159, 0.2924, 0.5764,
        0.2590, 0.2966, 0.3050, 0.3048, 0.2671, 0.5142, 0.2506, 0.5512, 0.4429,
        0.3305, 0.2661, 0.2780, 0.2794, 0.3353, 0.2623, 0.2642, 0.4534, 0.5134,
        0.2476, 0.2398, 0.2114, 0.2281, 0.2920, 0.2781, 0.2715, 0.5510, 0.2602,
        0.2002, 0.2554, 0.2483, 0.3358, 0.2994, 0.3137, 0.2697, 0.6491, 0.2838,
        0.5981, 0.2839, 0.2649, 0.3836, 0.2026, 0.2608, 0.2236, 0.5399, 0.2768,
        0.1798], device='cuda:0', grad_fn=<ViewBackward0>)
C(x,l):

tensor([[0.2301, 0.2473, 0.2301,  ..., 0.52

In practice, `ltn.diag` is designed to be used with quantifiers.
Every quantifier automatically calls `ltn.undiag` after the aggregation is performed, so that the variables keep their normal
behavior outside of the formula.
Therefore, it is recommended to use `ltn.diag` only in quantified formulas as follows.

Notice the call to `ltn.undiag` by the quantifier. After the quantification has been performed, the two variables restore their original labels.

实际上，`ltn.diag` 设计为与量词一起使用。
每个量词在聚合完成后会自动调用 `ltn.undiag`，以便变量在公式之外保持其正常行为。
因此，建议仅在量化公式中使用 `ltn.diag`，如下所示。

请注意量词对 `ltn.undiag` 的调用。在量化完成后，这两个变量会恢复它们原始的标签。

In [61]:
x, l = ltn.diag(x, l)
print(x.free_vars)
print(l.free_vars)
print(Forall([x, l], C(x, l)).value) # Aggregates only on the 100 "zipped" pairs. # 仅在100个“zipped”对上进行聚合
                                    # Automatically calls `ltn.undiag` so the behavior of x/l is unchanged outside of this formula. # 自动调用`ltn.undiag`，因此x/l的行为在此公式之外不变
print(x.free_vars)
print(l.free_vars)

['diag_x_l']
['diag_x_l']
tensor(0.3215, device='cuda:0', grad_fn=<RsubBackward1>)
['x']
['l']


## Guarded Quantifiers

One may wish to quantify over a set of elements whose grounding satisfy some **boolean** condition.

Let us assume $x$ is a variable from some domain and $m$ is a mask function that returns boolean values (that is, $0$ or $1$, not continuous truth degrees in $[0,1]$) for each element of the domain.

In guarded quantification, one has quantifications of the form:

- $(\forall x: m(x)) \text{ } \phi(x)$
which means "every x satisfying $m(x)$ also satisfies $\phi(x)$";
- $(\exists x: m(x)) \text{ } \phi(x)$
which means "some x satisfying $m(x)$ also satisfies $\phi(x)$".

The mask $m$ can also depend on other variables in the formula. For instance, the quantification $\exists y (\forall x:m(x,y)) \text{ } \phi(x,y)$ is also a valid sentence.

Let us consider the following example, that states that there exists an euclidian distance $d$ below which all pairs of points $x$, $y$ should be considered as similar:
$\exists d \ (\forall x,y : \mathrm{dist}(x,y) < d) \ ( \mathrm{Eq}(x,y))) $.

In this example, $Eq$ is a predicate measuring the similarity between two points, while $dist$ is a function which computes the euclidean distance between two points.

## 受限量词

我们可能希望对一组满足某个**布尔**条件的元素进行量化。

假设 $ x $ 是某个域中的变量， $ m $ 是一个掩码函数，对于域中的每个元素返回布尔值（即 $ 0 $ 或 $ 1 $，而不是 $ [0,1] $ 范围内的连续真值）。

在受限量化中，量化形式如下：

- $ (\forall x: m(x)) \text{ } \phi(x) $
  这意味着“每个满足 $ m(x) $ 的 $ x $ 也满足 $ \phi(x) $”；
- $ (\exists x: m(x)) \text{ } \phi(x) $
  这意味着“某个满足 $ m(x) $ 的 $ x $ 也满足 $ \phi(x) $”。

掩码 $ m $ 还可以依赖于公式中的其他变量。例如，量化 $ \exists y (\forall x:m(x,y)) \text{ } \phi(x,y) $ 也是一个有效的句子。

让我们考虑以下示例，该示例表明存在一个欧几里得距离 $ d $，在该距离以下的所有点对 $ x $ 和 $ y $ 应被视为相似：$\exists d \ (\forall x,y : \mathrm{dist}(x,y) < d) \ ( \mathrm{Eq}(x,y))) $。

在这个示例中，$ Eq $ 是一个谓词，用于衡量两点之间的相似性，而 $ dist $ 是一个函数，用于计算两点之间的欧几里得距离。

In [62]:
Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity # 用欧氏距离来衡量相似度

points = torch.rand((50, 2)) # 50 values in [0,1]^2 # 50个二维点
x = ltn.Variable("x", points)
y = ltn.Variable("y", points)
d = ltn.Variable("d", torch.tensor([.1,.2,.3,.4,.5,.6,.7,.8,.9]))

In [63]:
print("x:")
print(x.value)
print("\n")

print("y:")
print(y.value)
print("\n")

print("d:")
print(d.value) # 输出的形状是torch.Size([9, 1])，是一个9x1的张量
print(d.value.shape)
print("\n")

print(torch.tensor([.1,.2,.3,.4,.5,.6,.7,.8,.9]).shape)

x:
tensor([[0.8269, 0.4893],
        [0.9951, 0.7433],
        [0.2598, 0.9689],
        [0.1182, 0.8369],
        [0.8177, 0.1712],
        [0.2495, 0.3370],
        [0.9626, 0.8842],
        [0.7261, 0.4102],
        [0.7221, 0.4018],
        [0.0073, 0.7704],
        [0.4290, 0.5139],
        [0.0960, 0.4582],
        [0.7975, 0.8307],
        [0.1289, 0.6693],
        [0.7915, 0.0733],
        [0.9146, 0.1445],
        [0.5355, 0.5297],
        [0.6547, 0.5636],
        [0.8119, 0.0550],
        [0.6082, 0.9271],
        [0.4302, 0.9788],
        [0.0971, 0.5824],
        [0.4231, 0.5616],
        [0.1314, 0.6437],
        [0.6242, 0.6696],
        [0.6673, 0.5240],
        [0.3429, 0.6489],
        [0.7702, 0.1145],
        [0.0595, 0.9347],
        [0.3206, 0.1307],
        [0.6015, 0.3009],
        [0.6123, 0.9127],
        [0.4159, 0.1112],
        [0.1637, 0.0116],
        [0.9166, 0.0775],
        [0.0483, 0.2078],
        [0.9203, 0.0730],
        [0.4405, 0.1418],
        [

In [64]:
dist = lambda x, y: torch.unsqueeze(torch.norm(x.value - y.value, dim=1), 1) # function measuring euclidian distance # 用欧氏距离来衡量相似度
# torch.unsqueeze(..., 1)在计算出的结果的第一个维度添加一个新的维度，使其成为列向量（保持张量的形状）。
Exists(d,
      Forall([x, y],
            Eq(x, y),
            cond_vars=[x, y, d],
            cond_fn=lambda x, y, d: dist(x, y) < d.value
            )).value # 在具体实现中，dist(x, y) < d.value，应该是将每一组x和y都单独拎出来，去和一个d的值进行比较。d这里虽然有多个值，但是是因为最外面还有一个Exists。

tensor(0.7564, device='cuda:0', dtype=torch.float64)

In [65]:
print("Eq(x,y):"+"\n")
print(Eq(x, y).value)
print(Eq(x, y).value.shape)
print("\n")

print("d:"+ "\n")
print(d.value)
print(d.value.shape)
print("\n")

print("dist(x,y):"+"\n")
print(dist(x, y)) # 这里之所以输出的张量中，的50个元素，每个都是0.0，是因为x，y中的50个点是相同的，所以他们之间的距离都是0
print(dist(x, y).shape)
print("\n")

d2 = ltn.Constant(torch.tensor(.5))
d3 = ltn.Constant(torch.tensor([.3]))
print(d2.value)
print(d2.value.shape)
print(d3.value)
print(d3.value.shape)
Forall([x, y],
            Eq(x, y),
            cond_vars=[x, y],
            cond_fn=lambda x, y: dist(x, y) < d2.value
            ) # 这里用d2可以，用d3也可以。

Eq(x,y):

tensor([[1.0000, 0.7374, 0.4758,  ..., 0.5522, 0.7183, 0.5270],
        [0.7374, 1.0000, 0.4634,  ..., 0.4074, 0.6638, 0.4946],
        [0.4758, 0.4634, 1.0000,  ..., 0.3782, 0.6618, 0.8845],
        ...,
        [0.5522, 0.4074, 0.3782,  ..., 1.0000, 0.4926, 0.4272],
        [0.7183, 0.6638, 0.6618,  ..., 0.4926, 1.0000, 0.7280],
        [0.5270, 0.4946, 0.8845,  ..., 0.4272, 0.7280, 1.0000]],
       device='cuda:0')
torch.Size([50, 50])


d:

tensor([[0.1000],
        [0.2000],
        [0.3000],
        [0.4000],
        [0.5000],
        [0.6000],
        [0.7000],
        [0.8000],
        [0.9000]], device='cuda:0')
torch.Size([9, 1])


dist(x,y):

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

LTNObject(value=tensor(0.7243, device='cuda:0', dtype=torch.float64), free_vars=[])

As showed in the example, in order to use a guarded quantification it is enough to specify some condition variables (`cond_vars` parameter) and a
condition function (`cond_fn` parameter). In particular, `cond_vars` requires a list of LTN variables, while
`cond_fn` requires a function. The function computes a boolean mask, then the mask is used to select the values on which
the aggregation has to be computed.

In this specific example, the guarded quantification is used for aggregating over pairs of points whose distance is under a certain
threshold, specified by variable $d$. All the other pairs of points are not considered during the aggregation.

The guarded option is particularly useful to propagate gradients (see notebook on learning) only over a subset of the
domains, namely the domains which verifies the condition $m$.


如示例所示，为了使用受限量化，只需指定一些条件变量 (`cond_vars` 参数) 和一个条件函数 (`cond_fn` 参数)。特别地，`cond_vars` 需要一个 LTN 变量的列表，而 `cond_fn` 需要一个函数。该函数计算一个布尔掩码，然后使用该掩码选择需要计算聚合值的变量。

在这个具体示例中，受限量化用于聚合距离在某个阈值以下的点对，由变量 $d$ 指定。所有其他点对在聚合过程中不会被考虑。

受限选项特别有用，可以仅在满足条件 $m$ 的域的子集上传播梯度（参见学习相关的笔记本）。